In [ ]:
import tensorflow as tf
import numpy as np
from lhrNetUtils.data_loader import lhrNetDataLoader

from lhrNetUtils import Config

In [ ]:
config = Config("config.json")

In [ ]:
testDataLoader = lhrNetDataLoader("data/test_details.csv",8)
trainDataLoader = lhrNetDataLoader("data/train_details.csv",8)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32,64)),
    tf.keras.layers.Dense(2048,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256,activation="relu"),
    tf.keras.layers.Dense(8,activation="softmax")
])

loss_fn = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer="adam",loss=loss_fn,metrics=["accuracy"])

In [ ]:
model.fit(trainDataLoader,epochs=30)

In [ ]:
result = model.evaluate(testDataLoader)
dict(zip(model.metrics_names, result))

In [ ]:
np.set_printoptions(suppress=True)

prediction_probs = model.predict(np.array([np.genfromtxt("data/1731742593.csv" ,delimiter=",")]))
display(prediction_probs)

prediction = int(prediction_probs.argmax())

print(f"Prediction is \"{config.get_states()[prediction]}\" with {prediction_probs[0][prediction] * 100}% probablity")